# Association Rule Mining using FP-Growth

This notebook demonstrates how to extract frequent itemsets and association rules from transaction data using the FP-Growth algorithm, inspired by the Kaggle notebook by Mohammed Derouiche.

---

**Dataset**: Cleaned version of the [UCI Online Retail Dataset](https://archive.ics.uci.edu/ml/datasets/online+retail)  
**Goal**: Generate association rules in the form `antecedents → consequents` to be used in a recommendation system API.


In [1]:
# Import necessary libraries
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [2]:
# Load the dataset
df = pd.read_csv("../data/transactions_fpgrowth.csv")

In [3]:
# Convert the 'items' column to a list of transactions
transactions = df["items"].str.split(",")

In [4]:
# Transform the transactions into a one-hot encoded DataFrame
te = TransactionEncoder()
te_data = te.fit_transform(transactions)
df_trans = pd.DataFrame(te_data, columns=te.columns_)

In [5]:
# Apply the FP-Growth algorithm to find frequent itemsets
frequent_itemsets = fpgrowth(df_trans, min_support=0.01, use_colnames=True)

In [6]:
# Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [7]:
# Resulting rules sorted by lift
rules.sort_values("lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
590,"(coffee, sugar)",(set 3 retrospot tea),0.013005,0.013005,0.013005,1.000000,76.896266,1.0,0.012835,inf,1.000000,1.000000,1.000000,1.000000
591,(set 3 retrospot tea),"(coffee, sugar)",0.013005,0.013005,0.013005,1.000000,76.896266,1.0,0.012835,inf,1.000000,1.000000,1.000000,1.000000
593,(sugar),"(set 3 retrospot tea, coffee)",0.013005,0.013005,0.013005,1.000000,76.896266,1.0,0.012835,inf,1.000000,1.000000,1.000000,1.000000
588,"(set 3 retrospot tea, coffee)",(sugar),0.013005,0.013005,0.013005,1.000000,76.896266,1.0,0.012835,inf,1.000000,1.000000,1.000000,1.000000
585,(sugar),(set 3 retrospot tea),0.013005,0.013005,0.013005,1.000000,76.896266,1.0,0.012835,inf,1.000000,1.000000,1.000000,1.000000
584,(set 3 retrospot tea),(sugar),0.013005,0.013005,0.013005,1.000000,76.896266,1.0,0.012835,inf,1.000000,1.000000,1.000000,1.000000
907,(regency tea plate pink),(regency tea plate green),0.012087,0.014569,0.010900,0.901786,61.895899,1.0,0.010724,10.033475,0.995881,0.691781,0.900334,0.824967
906,(regency tea plate green),(regency tea plate pink),0.014569,0.012087,0.010900,0.748148,61.895899,1.0,0.010724,3.922595,0.998390,0.691781,0.745067,0.824967
583,(sugar),(coffee),0.013005,0.017213,0.013005,1.000000,58.094044,1.0,0.012781,inf,0.995736,0.755486,1.000000,0.877743
592,(coffee),"(set 3 retrospot tea, sugar)",0.017213,0.013005,0.013005,0.755486,58.094044,1.0,0.012781,4.036558,1.000000,0.755486,0.752264,0.877743
